In [10]:
import numpy as np
import open3d as o3d

def create_coordinate_frame(size=100.0):
    """创建坐标系模型"""
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)

def visualize_point_cloud_with_frame():
    # 1. 读取点云
    # pcd = o3d.io.read_point_cloud("thickboard.pcd")  # 替换为你的点云文件
    pcd = o3d.io.read_point_cloud("workoiece_frame.pcd")

        
    # 2. 创建坐标系
    coordinate_frame = create_coordinate_frame()
    
    # 3. 可视化点云和坐标系
    o3d.visualization.draw_geometries([pcd, coordinate_frame],
                                    window_name="Point Cloud with Coordinate Frame",
                                    width=800,
                                    height=600,
                                    left=50,
                                    top=50,
                                    point_show_normal=False,
                                    mesh_show_wireframe=False,
                                    mesh_show_back_face=False)

def visualize_with_multiple_frames():
    """显示多个位置的坐标系"""
    # 1. 读取点云
    # pcd = o3d.io.read_point_cloud("thickboard.pcd")
    pcd = o3d.io.read_point_cloud("workoiece_frame.pcd")

    points = np.asarray(pcd.points)

    # 获取点云范围
    min_bound = points.min(axis=0)  # 最小xyz值
    max_bound = points.max(axis=0)  # 最大xyz值
    center = (min_bound + max_bound) / 2  # 点云中心
    
    # 2. 创建坐标系列表
    frames = []
    # 原点坐标系
    frames.append(create_coordinate_frame())
    
    # 点云中心坐标系
    center_frame = create_coordinate_frame()
    min_frame = create_coordinate_frame()
    max_frame = create_coordinate_frame()
    points = np.asarray(pcd.points)
    center = np.mean(points, axis=0)
    # 将坐标系平移到点云中心
    center_frame.translate(center)
    min_frame.translate(min_bound)
    max_frame.translate(max_bound)

    frames.append(center_frame)
    # frames.append(min_frame)
    # frames.append(max_frame)
    
    # 3. 可视化
    o3d.visualization.draw_geometries([pcd] + frames,
                                    window_name="Point Cloud with Multiple Frames",
                                    width=800,
                                    height=600)

if __name__ == "__main__":
    # 基础版本
    visualize_point_cloud_with_frame()
    
    # 显示多个坐标系的版本
    # visualize_with_multiple_frames()

[Open3D WARNING] Read PCD failed: unable to open file: workoiece_frame.pcd
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [11]:
import numpy as np
import open3d as o3d

# 读取点云
pcd = o3d.io.read_point_cloud("thickboard.pcd")
points = np.asarray(pcd.points)

# 获取点云范围
min_bound = points.min(axis=0)  # 最小xyz值
max_bound = points.max(axis=0)  # 最大xyz值
center = (min_bound + max_bound) / 2  # 点云中心

print("点云范围：")
print(f"最小边界: {min_bound}")
print(f"最大边界: {max_bound}")
print(f"点云中心: {center}")
# 创建坐标系
coordinate_frame = create_coordinate_frame(size=50.0)

# 将坐标系平移到点云中心
# coordinate_frame.translate(max_bound)

# 可视化点云和坐标系
o3d.visualization.draw_geometries([pcd, coordinate_frame],
                                  window_name="Point Cloud with Coordinate Frame",
                                  width=800,
                                  height=600,
                                  left=50,
                                  top=50,
                                  point_show_normal=False,
                                  mesh_show_wireframe=False,
                                  mesh_show_back_face=False)

点云范围：
最小边界: [-117.99971771  -10.10173416  440.0020752 ]
最大边界: [ 48.43133926 165.66114807 537.33026123]
点云中心: [-34.78418922  77.77970695 488.66616821]


In [24]:
import numpy as np
import open3d as o3d

# 读取点云
# pcd = o3d.io.read_point_cloud("thickboard.pcd")  # 替换为你的点云文件路径
pcd = o3d.io.read_point_cloud("workpiece_frame.pcd")
# pcd = o3d.io.read_point_cloud("thickBoardweld.pcd")

# 创建坐标系
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50.0,    # 坐标轴的长度
    origin=[0, 0, 0]  # 坐标系原点位置
)

# 创建可视化器
vis = o3d.visualization.Visualizer()
vis.create_window()

# 添加点云和坐标系到可视化器
vis.add_geometry(pcd)
vis.add_geometry(mesh_frame)

# 设置初始视角（可选）
opt = vis.get_render_option()
opt.background_color = np.asarray([255, 255, 255])  # 设置背景颜色为黑色
opt.point_size = 1.0  # 设置点的大小

# 运行可视化
vis.run()

# 关闭窗口
vis.destroy_window()

In [ ]:
import numpy as np
import open3d as o3d
import ipywidgets as widgets
from IPython.display import display

# 读取点云
pcd = o3d.io.read_point_cloud("workpiece_frame.pcd")

# 创建坐标系
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50.0,    # 坐标轴的长度
    origin=[0, 0, 0]  # 坐标系原点位置
)

# 创建可视化器
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()

# 添加点云和坐标系到可视化器
vis.add_geometry(pcd)
vis.add_geometry(mesh_frame)

# 设置初始视角（可选）
opt = vis.get_render_option()
opt.background_color = np.asarray([255, 255, 255])  # 设置背景颜色为白色
opt.point_size = 1.0  # 设置点的大小

# 定义切除函数
def update_point_cloud(x_min, x_max):
    points = np.asarray(pcd.points)
    mask = (points[:, 0] >= x_min) & (points[:, 0] <= x_max)
    filtered_points = points[mask]
    
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
    
    vis.clear_geometries()
    vis.add_geometry(filtered_pcd)
    vis.add_geometry(mesh_frame)
    vis.poll_events()
    vis.update_renderer()

# 创建滑块
x_min_slider = widgets.FloatSlider(value=-50, min=-100, max=100, step=1, description='X Min')
x_max_slider = widgets.FloatSlider(value=50, min=-100, max=100, step=1, description='X Max')

# 定义滑块回调函数
def on_slider_change(change):
    x_min = x_min_slider.value
    x_max = x_max_slider.value
    update_point_cloud(x_min, x_max)

# 绑定滑块回调函数
x_min_slider.observe(on_slider_change, names='value')
x_max_slider.observe(on_slider_change, names='value')

# 显示滑块
display(x_min_slider, x_max_slider)

# 运行可视化
vis.run()

# 关闭窗口
vis.destroy_window()

FloatSlider(value=-50.0, description='X Min', min=-100.0, step=1.0)

FloatSlider(value=50.0, description='X Max', min=-100.0, step=1.0)

In [ ]:
import numpy as np
import open3d as o3d
import ipywidgets as widgets
from IPython.display import display

# 读取点云
pcd = o3d.io.read_point_cloud("workpiece_frame.pcd")

# 创建坐标系
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50.0,    # 坐标轴的长度
    origin=[0, 0, 0]  # 坐标系原点位置
)

# 创建可视化器
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()

# 添加点云和坐标系到可视化器
vis.add_geometry(pcd)
vis.add_geometry(mesh_frame)

# 设置初始视角（可选）
opt = vis.get_render_option()
opt.background_color = np.asarray([255, 255, 255])  # 设置背景颜色为白色
opt.point_size = 1.0  # 设置点的大小

# 定义切除函数
def update_point_cloud(x_min, x_max):
    points = np.asarray(pcd.points)
    mask = (points[:, 0] >= x_min) & (points[:, 0] <= x_max)
    filtered_points = points[mask]
    
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
    
    vis.clear_geometries()
    vis.add_geometry(filtered_pcd)
    vis.add_geometry(mesh_frame)
    vis.poll_events()
    vis.update_renderer()

# 创建滑块
x_min_slider = widgets.FloatSlider(value=-50, min=-100, max=100, step=1, description='X Min')
x_max_slider = widgets.FloatSlider(value=50, min=-100, max=100, step=1, description='X Max')

# 定义滑块回调函数
def on_slider_change(change):
    x_min = x_min_slider.value
    x_max = x_max_slider.value
    update_point_cloud(x_min, x_max)

# 绑定滑块回调函数
x_min_slider.observe(on_slider_change, names='value')
x_max_slider.observe(on_slider_change, names='value')

# 显示滑块
display(x_min_slider, x_max_slider)

# 运行可视化
vis.run()